#### Functions (IGNORE)

##### Functions for Downloading Data
- 'get_data_from_start_to_end': Load in the data from YFinance API
- 'download_stock_data_for_periods': Look at different periods to understand how technical analysis changes throughout changes in macroeconomy 

In [1]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [2]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [3]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

##### Descriptive Analytics

In [4]:
# create a function to get a table of descriptive analysis of the different sector etfs in different time periods

# import pandas 
import pandas as pd

def create_analysis(tickers,periods,nested_dict,col_name,function_name):
    """
    parameters:

    tickers: the list of tickers that will be looped through
    periods: the list of periods that will be looped through
    nested_dict: the dictionary which can be accessed through nested_dict[period][ticker]
    col_name: the column of the nested dictionary
    function_name: the function that you want to be applied

    returns:
    a dataframe with columns as time periods and rows as sector etfs with the column name within the nested dictionary with a descriptive statistic applied
    """
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = nested_dict[period][ticker][col_name]
            df.at[ticker, period] = getattr(pd.Series(data), function_name)()
    return df

##### Visualising Data

In [5]:
import plotly.graph_objects as go

def visualize_candlestick(data, title="Candlestick Chart"):
    """
    Visualizes candlestick data using Plotly.

    Parameters:
    data (pd.DataFrame): The dataframe containing the stock's OHLC data. 
                         It should contain 'Open', 'High', 'Low', and 'Close' columns.
    title (str): The title of the plot.

    Returns:
    fig: The Plotly figure object.
    """
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])

    # Set titles and labels
    fig.update_layout(title=title,
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)
    
    fig.show()

    return fig

##### Stock Trading Strategies

In [6]:
import numpy as np

def buy_hold_roi(nested_dict, periods, tickers, column):
    """
    Objective: 
    To return the ROI when purchasing stocks during different economic time periods.
    
    Parameters:
    nested_dict: holds the historical data
    periods: a list of time periods to be used as columns
    tickers: a list of tickers to be used as rows

    Returns:
    DataFrame with the ROI using a buy and hold strategy.
    """
    # Create a DataFrame to give the ROI for different investments
    roi = pd.DataFrame(index=tickers, columns=periods)

    # Sort through each ticker and each period
    for ticker in tickers:
        for period in periods:
            # Get the DataFrame through the nested dictionary
            data = nested_dict.get(period, {}).get(ticker)

            # Make sure that the data is present
            if data is not None and column in data.columns:
                start_price = data[column].iloc[0]
                end_price = data[column].iloc[-1]

                start_date = data.index[0]
                end_date = data.index[-1]

                num_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

                if num_days > 0:  # Ensure we don't divide by zero
                    # Calculate percentage difference 
                    total_return = ((end_price - start_price) / start_price) * 100
                    annualized_roi = ((1 + total_return / 100) ** (365 / num_days) - 1) * 100  # Annualized percentage
                    roi.at[ticker, period] = annualized_roi
                else:
                    roi.at[ticker, period] = np.nan
            else:
                roi.at[ticker, period] = np.nan

    return roi


In [7]:
import random

def stochastic_modeling(nested_dict, tickers, periods,num_samples):
    # Store the returns in a nested dictionary
    nested_dict_returns = {period: {ticker: [] for ticker in tickers} for period in periods}

    # Go through each economic time period
    for period in periods:
        max_index = len(nested_dict[period]) - 30  # Ensure there's enough data to calculate ROI

        # Generate random samples from the valid range
        random_dates = random.choices(range(max_index), k=num_samples)

        for ticker in tickers:
            for date_idx in random_dates:
                start_price = nested_dict[period][ticker].iloc[date_idx]
                end_price = nested_dict[period][ticker].iloc[date_idx + 30]

                # Get the return by the Holding Period Return
                roi = ((end_price - start_price) / start_price) * 100

                nested_dict_returns[period][ticker].append(roi)

    return nested_dict_returns  # Return the nested dictionary with returns

In [8]:
def stochasting_roi(tickers,periods,return_rates_list):
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = return_rates_list[period][ticker]
            df.at[ticker,period] = sum(data)/len(data)
    return df

# EDA and Data Cleaning

#### Assignment: Exploratory Data Analysis (EDA) and Data Cleaning Report

## Dataset Overview:

### Yahoo Finance API

The data that will be used for my capstone can be accessed through the Yahoo Finance API.

The Yahoo Finance API is a free popular API that is used by developers to load in a variety of stock data. This includes financial, candlestick and other information that can be utilized to make informed investment decisions.

Yahoo Finance is free and very easy to set up. First pip install yfinance and then simply load it into your notebook.


In [9]:
# load the API in
import yfinance as yf # yf is the usual name for the package

As an example load in Microsoft data from the YFinance API.

In [10]:
# save the ticker information of 'MSFT' as a variable
msft = yf.Ticker('MSFT')

# .info can be used for information about the company
print(msft.info)

# .get_balance_sheets returns the balance sheet information
print(msft.get_balance_sheet())

{'address1': 'One Microsoft Way', 'city': 'Redmond', 'state': 'WA', 'zip': '98052-6399', 'country': 'United States', 'phone': '425 882 8080', 'website': 'https://www.microsoft.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and pow

The list of functions that are associated to gain data on specific stocks are seemingly endless and should be the only data source that is going to be used for my Capstone Project.

### Stock Data
There are thousands of equities to choose from on the stock market, to reduce the overload of data that will be used in training the trading techniques sector ETF's will be used instead.

#### Exchange Traded Funds (ETF)
Electronically Traded Funds or ETF's are an investment equity which has a basket of securities that tracks or seeks to outperform indexes. It takes a smaller proportion of a large number of individual stocks so that the equity is a broader representation of a specific idea. For example if you want to invest in small-cap companies, there are a number of ETF's that can be used to take a smaller investment of a hundred companies rather than one. This looks to spread out investments to reduce risk.

#### Economic Time Periods
The macroeconomic business cycle is the representation of overall economic activity within the economy. During different periods of economic activity the market sectors will react differently. The following are the typical components of the macroeconomic business cycle.

- Trough : This is the low point of the economy where supply is in excess and demand is in shortage. This can turn into a recession which is where there are two consecutive quarters where the GDP growth is negative.

- Expansion : Following a trough, there is a period of economic growth which leads to an increase in consumer and business confidence. This will result in an increase in wage growth and a decrease in unemployment. Expansion periods should be steady and not exponential to ensure prolonged prosperity.

- Peak : A peak occurs when an expansion has reached its limit and there ends up being downward pressure on the growth rate within economic activity. Businesses will then reevaluate their decision making to review budgets and other outlooks.

- Contraction : This is when growth slows, unemployment increases and business investment falls. 


In [11]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction']

#### GICS Sectors
According to Global Industry Classification Standard there are 11 major sectors at play within the economy. Each with their own industries and sub-industries but this capstone project there will be a concentration on the sectors. 

For the capstone project a concentration on ETF's that represent the sectors will be used to help understand how different sectors will perform. The following ETF equities are going to be invested in based on their sector. More information about the equities can be found in the report.

- Materials : XLB 
- Industrials : XLI
- Financials : XLF
- Information Technology : XLK
- Consumer Discretionary : XLY
- Consumer Staples : XLP
- Energy : XLE
- Healthcare : XLV
- Communication Services : VOX
- Utilities : XLU
- Real Estate : IYR

In [12]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

### Nested Dictionary Storage of Relevant Data
Using the YFinance API load in candlestick data with the 11 industry sectors during the four different economic time periods. This can be used to evaluate sector etfs during different economic time periods using the daily adjusted closed price.

In [13]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

The layout of the nested dictionary means that for any macroeconomic business cycle the candlestick data for any of the 11 sector ETF's as listed above can be accessed. 

The following is an example of how you can access different data within the nested dictionary.

In [14]:
# access healthcare sector during an expansion
# first load in the trough data
trough_sector_data = sector_etf_data['trough']

In [15]:
# now access the healthcare sector etf 'XLV'
trough_sector_data['XLV'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-10-01,30.100000,30.480000,30.100000,30.250000,23.012999,6053600
2008-10-02,30.250000,30.590000,29.930000,30.299999,23.051035,6353400
2008-10-03,30.600000,30.600000,29.650000,29.650000,22.556541,6814400
2008-10-06,29.400000,29.879999,27.410000,28.540001,21.712099,8545000
2008-10-07,28.719999,28.780001,27.389999,27.850000,21.187178,5060200


This returns the candlestick data for the XLV equity during a trough which ranged from 2008-01-02 to 2009-6-30 due to the Global Financial Crises. 

The same process can be completed in one step.

In [16]:
# access 'XLV' equity in one line of code
sector_etf_data['trough']['XLV'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-10-01,30.100000,30.480000,30.100000,30.250000,23.012999,6053600
2008-10-02,30.250000,30.590000,29.930000,30.299999,23.051035,6353400
2008-10-03,30.600000,30.600000,29.650000,29.650000,22.556541,6814400
2008-10-06,29.400000,29.879999,27.410000,28.540001,21.712099,8545000
2008-10-07,28.719999,28.780001,27.389999,27.850000,21.187178,5060200


The columns contain candlestick data which is:

- 'Open' : The price of the equity at the beginning of the trading day.
- 'High' : The highest price of the equity throughout the day.
- 'Low' : The lowest price of the equity throughout the day.
- 'Close' : The price of the equity at the end of the trading day
- 'Adjusted Close' : The came as closing price however it takes dividends, stock splits and new stock offerings into account
- 'Volume': The number of times that equity is traded within a day.

This is extremely useful for technical analysis which is heavily reliant on historical data. There is now an easy process to access any stock data's candlestick data (in this case sector ETF's) during any time period (different macroeconomic time periods). This is going to be used to as a basis for the capstone project. 

There are currently 6 columns however this will grow as different techniques require new columns to be used as features. 

The size of the data currently stands at: 4 time periods with 11 sectors with 6 columns which is 264 columns * (150 days for trough + 375 days for contraction + 2180 days for expansion + 169 for peak) = 758,736

## Initial Data Inspection / Univariate Analysis
The dataset is a nested dictionary with a number of time series. Using typical descriptive statistics might not be as relevant. Using the create_analysis function you are able to load in the nested dictionary and apply different descriptive statistical reports on the different sector etfs during different time periods as a table. This can be adjusted to fit a number of descriptive statistics and for different parameters within the dataframe of the nested dictionary.

### Numerical Analysis

In [17]:
# get the mean value of these stocks during the different time periods
mean_data = create_analysis(sector_etf_tickers,economic_cycle_periods_list,sector_etf_data,'Adj Close','mean')
mean_data

,trough,expansion,peak,contraction
XLB,16.686582,33.341454,52.917294,28.608563
XLI,15.852843,36.435797,72.431897,26.289616
XLF,6.886585,12.601652,25.941764,14.533609
XLK,12.620077,28.430873,79.864512,18.592997
XLY,16.963294,49.350607,116.16315,25.01046
XLP,14.870896,29.717788,53.480787,17.960931
XLE,28.421203,53.23693,47.020173,46.356162
XLV,19.386921,41.449804,87.094059,24.519836
VOX,31.093878,62.755677,85.677095,44.45534
XLU,15.769571,26.847841,53.767981,22.055005


In [18]:
# get the median value of these stocks
median_data = create_analysis(sector_etf_tickers,economic_cycle_periods_list,sector_etf_data,'Adj Close','median')
median_data

,trough,expansion,peak,contraction
XLB,16.347618,32.295511,52.799816,28.447906
XLI,16.113988,35.817591,71.974785,26.422621
XLF,6.943471,13.03797,25.566185,14.805957
XLK,12.470977,27.199531,77.621567,18.33905
XLY,16.923295,50.454151,116.364227,25.156828
XLP,14.960623,30.248541,53.487614,18.002188
XLE,28.328449,53.228474,46.977001,45.894325
XLV,19.492294,41.068962,84.821175,24.091341
VOX,30.99697,64.17902,84.958977,43.722992
XLU,15.865283,26.341281,53.986683,22.073484


In [19]:
# get the difference between these two to decide if the mean and median are close
mean_data-median_data

,trough,expansion,peak,contraction
XLB,0.338964,1.045943,0.117478,0.160657
XLI,-0.261144,0.618206,0.457112,-0.133005
XLF,-0.056886,-0.436318,0.375579,-0.272348
XLK,0.1491,1.231342,2.242945,0.253947
XLY,0.039999,-1.103544,-0.201077,-0.146368
XLP,-0.089727,-0.530753,-0.006827,-0.041256
XLE,0.092753,0.008456,0.043172,0.461837
XLV,-0.105373,0.380842,2.272885,0.428495
VOX,0.096908,-1.423343,0.718118,0.732348
XLU,-0.095712,0.50656,-0.218702,-0.01848


Notice that the mean and median values are very close together which means that the typical value of a stock is likely normally distributed where there are data values that above and below the average with common occurence.

In [20]:
# how does the mean value change during the different states of the macroeconomic cycle
mean_data.mean()

trough         17.959496
expansion      38.249084
peak           68.625438
contraction    27.633456
dtype: float64

Observe that the mean values of these sector etf's follow what you would expect. There are low values during the trough and contraction with higher values taking place during the expansion and peak. This suggests that the sector etfs strongly match the macroeconomic time periods that are taking place.

In [21]:
# get the standard deviation
create_analysis(sector_etf_tickers,economic_cycle_periods_list,sector_etf_data,'Adj Close','std')

,trough,expansion,peak,contraction
XLB,1.8369,4.834145,1.567754,1.689659
XLI,1.812336,6.592581,3.025955,1.449261
XLF,1.60123,2.353624,1.424399,1.822995
XLK,0.955407,4.112598,6.027886,1.242621
XLY,1.718796,9.03084,3.309264,1.311233
XLP,0.887215,4.054363,1.675482,0.437353
XLE,2.229603,6.899179,1.668784,3.921772
XLV,1.066655,9.149629,4.829239,1.223715
VOX,2.207266,8.344902,3.730776,3.540011
XLU,1.029368,3.061783,2.171691,1.239613


In [22]:
# get the min for each stock
create_analysis(sector_etf_tickers,economic_cycle_periods_list,sector_etf_data,'Adj Close','min')

,trough,expansion,peak,contraction
XLB,12.936709,25.42024,48.822544,23.116423
XLI,11.310508,26.41921,66.340538,22.242884
XLF,3.780497,8.54141,23.495203,10.250641
XLK,10.512967,21.661146,67.064911,15.143781
XLY,13.169909,33.771393,104.762817,21.54611
XLP,12.749035,22.914076,48.905949,17.105053
XLE,23.203623,39.914349,43.396389,36.121544
XLV,16.746851,28.426874,79.929443,22.719435
VOX,24.479897,47.67556,77.397774,36.699162
XLU,13.078053,22.511066,49.740742,18.57752


In [23]:
# get the max for each stock
create_analysis(sector_etf_tickers,economic_cycle_periods_list,sector_etf_data,'Adj Close','max')

,trough,expansion,peak,contraction
XLB,23.222456,41.566235,56.028221,32.200748
XLI,21.929296,48.189205,78.330193,28.96306
XLF,12.458467,16.926834,28.394543,18.533949
XLK,15.677787,37.494659,93.997902,21.672705
XLY,22.636202,65.23307,123.115746,27.893873
XLP,18.105206,38.393639,56.881184,18.856043
XLE,37.702049,68.046036,50.070969,54.416744
XLV,23.051035,60.622299,97.517143,27.530712
VOX,36.781956,76.10437,94.986649,53.203087
XLU,18.792997,35.942589,59.973793,24.436256


### Visualisation
Creating a dashboard that can be used to adjust different time periods/stocks to visualise your stock data through candlestick data would be extremely useful. This will be done later. For now use the 'visualize_candlestick' function to plot candlestick data.

In [24]:
# create a sample visualisation of 'XLV' healthcare ETF from 2009-06 to 2010-01-01
for period in economic_cycle_periods_list:
    visualize_candlestick(sector_etf_data[period]['XLV'],f'XLK {period}')

By contrasting the difference in visualisation of the data it is clear that the 'XLK' or sector etf follows a movement similar to the state of the economy. 

## Stock Investment: Buy and Hold
An extremely popular type of investment is a passive one, this is where individuals will see an opportunity for long term growth of an equity and will simply purchase the stock and hold onto it for a varied period of time. As the beginning component of analysis, use a buy and hold strategy to look at the different stocks during different time periods to determine yearly return on investment 

In [25]:
buy_hold_roi(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers,'Adj Close')

,trough,expansion,peak,contraction
XLB,-22.979012,14.676827,11.802993,-20.770517
XLI,-33.796268,20.419993,20.97598,-23.043295
XLF,-53.252205,25.012766,25.408895,-38.349874
XLK,-13.845848,19.182308,59.557905,-27.193473
XLY,-23.365377,24.228111,23.574788,-21.081064
XLP,-22.774007,17.478535,22.909509,-3.638288
XLE,-23.477634,5.689467,-7.317901,-15.718578
XLV,-19.875631,27.096299,24.263895,-18.57291
VOX,-8.725711,13.371271,26.766951,-33.074433
XLU,-24.913896,14.433957,31.706761,-24.29217


The drawback to this method is that it takes it at the start and holds it until the end. This is not useful as what if you bought on a particularly high or low day. 

During these macroeconomic cycles buy and hold for one month based on a random selection of start dates. This can then be used to find the distribution of returns which can then be used to look at expected returns and the standard deviations during this time.

In [26]:
# create an adjusted close price sector etf dictionary which can changed based on the macroeconomic period
adjusted_close_sector_etf = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

In [28]:
expected_values_sector_etf_buy_hold = stochastic_modeling(adjusted_close_sector_etf,sector_etf_tickers,economic_cycle_periods_list,1000)
stochasting_roi(sector_etf_tickers,economic_cycle_periods_list,expected_values_sector_etf_buy_hold)

,trough,expansion,peak,contraction
XLB,1.229318,1.698172,1.161418,-1.361557
XLI,-0.964014,2.242443,2.053317,-1.8909
XLF,-2.164951,2.63322,2.709001,-4.344845
XLK,1.292213,2.105935,4.579457,-2.785714
XLY,2.077073,2.502823,1.536101,-0.639505
XLP,-1.427398,2.04818,1.955246,0.051007
XLE,-0.015159,1.131881,-0.149514,-1.445213
XLV,-1.388068,2.982618,2.749358,-1.547645
VOX,2.36564,1.759194,2.523628,-3.36694
XLU,-1.41963,1.722576,2.408706,-2.792451
